# question 2

In [1]:
from os import path
import os
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import math

In [2]:
os.getcwd()

'/home/eason/109-2-OperationReseach/Case2'

In [3]:
work_dir = os.getcwd()
data_path = path.join(work_dir, 'data')
data_file = path.join(data_path, 's2.xlsx')

In [4]:
Demand_df = pd.read_excel(data_file, sheet_name='Demand', index_col='Product')
Init_df = pd.read_excel(data_file, sheet_name='Initial inventory', index_col='Product')
ShippingCost_df = pd.read_excel(data_file, sheet_name='Shipping cost', index_col='Product')
InTransit_df = pd.read_excel(data_file, sheet_name='In-transit', index_col='Product')
CBM_df = pd.read_excel(data_file, sheet_name='Size', index_col='Product')
PriceAndCost_df = pd.read_excel(data_file, sheet_name='Price and cost', index_col='Product')
Shortage_df = pd.read_excel(data_file, sheet_name='Shortage', index_col='Product')
VendorProduct_df = pd.read_excel(data_file, sheet_name='Vendor-Product', index_col='Product') - 1
VendorCost_df = pd.read_excel(data_file, sheet_name='Vendor cost', index_col='Vendor')
Bound_df = pd.read_excel(data_file, sheet_name='Bounds', index_col='Product')
Conflict_df = pd.read_excel(data_file, sheet_name='Conflict', index_col='Conflict') - 1

N, M = Demand_df.shape
K = 3 #(express delivery, air freight, Ocean freight)
V = VendorCost_df.shape[0]
ContainerCap = 30 #Case 1 data
ContainerCost = 2750 #Case 1 data

Demand_ij = Demand_df.to_numpy()
Init_i = Init_df.to_numpy().squeeze()
BuyCost_i = PriceAndCost_df['Purchasing cost'].to_numpy()
HoldCost_i = PriceAndCost_df['Holding cost'].to_numpy()
Transit_ij = InTransit_df.to_numpy()
ShipFixedCost_k = np.array([100, 80, 50]) #Case 1 data
ShipVarCost_ik = np.concatenate(( ShippingCost_df.to_numpy(), np.zeros((N,1)) ), axis=1)
CBM_i = CBM_df.to_numpy().squeeze()
LostSaleCost_i = Shortage_df['Lost sales'].to_numpy()
BackOrderCost_i = Shortage_df['Backorder'].to_numpy()
BackOrderProb_i = Shortage_df['Backorder percentage'].to_numpy()
VendorFixedCost_v = VendorCost_df.to_numpy().squeeze()
MinOrder_i = Bound_df.to_numpy().squeeze()
ConflictPair_alpha = Conflict_df.to_numpy()
ProductVendor_i = VendorProduct_df.to_numpy().squeeze()
M_big = np.sum(Demand_ij) + sum(Init_i)

In [5]:
#model
instance_name = 's2'
model = gb.Model(name=instance_name)

Academic license - for non-commercial use only - expires 2021-06-26
Using license file /home/eason/gurobi.lic


In [6]:
#decision variable
x = model.addVars(N, M, K, vtype=GRB.INTEGER,name='x')
Abin = model.addVars(M, K, vtype=GRB.BINARY, name='Abin')
ContainerCnt = model.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt')
StockLevel = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='StockLevel')
Shortage = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='Shortage')
Bbin = model.addVars(N, M, vtype=GRB.BINARY, name='Bbin')
Cbin = model.addVars(N, M, vtype=GRB.BINARY, name='Cbin')
Dbin = model.addVars(V, M, vtype=GRB.BINARY, name='Dbin')

In [7]:
#Expr
VolumeInOceanExpr_j = [
gb.LinExpr(
    quicksum(x[i,j,2] * CBM_i[i] for i in range(N))
)    
for j in range(M)]

BackOrderCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * BackOrderProb_i[i] )
    for j in range(M)]
for i in range(N)]

LostSaleCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * (1 - BackOrderProb_i[i]) )
    for j in range(M)]
for i in range(N)]

In [8]:
#obj function
TotalPurchaseCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for i in range(N))
        for j in range(M))
    for k in range(K))
)

TotalShipFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Abin[j,k] * ShipFixedCost_k[k]
        for k in range(K))
    for j in range(M))
)

TotalShipVarCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * ShipVarCost_ik[i,k]
            for k in range(K))
        for j in range(M))
    for i in range(N))
)

TotalHoldingCost = gb.LinExpr(
    quicksum(
        quicksum(
            StockLevel[i,j] * HoldCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalContainerCost = gb.LinExpr(
    quicksum(ContainerCnt[j] for j in range(M)) * ContainerCost
)

TotalBackOrderCost = gb.LinExpr(
    quicksum(
        quicksum(
            BackOrderCntExpr_ij[i][j] * BackOrderCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalLostSaleCost = gb.LinExpr(
    quicksum(
        quicksum(
            LostSaleCntExpr_ij[i][j] * LostSaleCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalVendorFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Dbin[v,j] * VendorFixedCost_v[v]
        for v in range(V))
    for j in range(M))
)

In [9]:
#set obj function
model.setObjective(
    TotalPurchaseCost +
    TotalShipFixedCost + 
    TotalShipVarCost + 
    TotalHoldingCost + 
    TotalContainerCost + 
    TotalBackOrderCost + 
    TotalLostSaleCost +
    TotalVendorFixedCost
)

In [10]:
#Contrain

#let Abin to behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for i in range(N)) / M_big <= Abin[j,k]
for j in range(M)
for k in range(K))

#let ContainerCnt to behave correctly
_ = model.addConstrs(
    VolumeInOceanExpr_j[j] / ContainerCap <= ContainerCnt[j]
for j in range(M))

#let Stocklevle & Shortage behave correctly
_ = model.addConstrs(StockLevel[i,0] - Shortage[i,0] == Init_i[i] - Demand_ij[i][0] for i in range(N))
_ = model.addConstrs(StockLevel[i,1] - Shortage[i,1] == StockLevel[i,0] + x[i,0,0] + Transit_ij[i][1] - Demand_ij[i][1] - Shortage[i,0] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,2] - Shortage[i,2] == StockLevel[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i][2] - Demand_ij[i][2] - Shortage[i,1] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,j] - Shortage[i,j] == StockLevel[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - Demand_ij[i][j] - Shortage[i,j-1] * BackOrderProb_i[i] for i in range(N) for j in range(3,M))

_ = model.addConstrs(StockLevel[i,j] <= M_big * (1-Bbin[i,j]) for i in range(N) for j in range(M))
_ = model.addConstrs(Shortage[i,j] <= M_big * Bbin[i,j] for i in range(N) for j in range(M))

#let Cbin behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for k in range(K)) / M_big <= Cbin[i,j]
for i in range(N)
for j in range(M))

#let Dbin behave correctly
_ = model.addConstrs(
    quicksum(
        quicksum(x[i,j,k] for k in range(K))
    for i in range(N) if ProductVendor_i[i] == v) / M_big <= Dbin[v,j]
for v in range(V)
for j in range(M))

#Minumin order Bound
_ = model.addConstrs(
    Cbin[i,j] * MinOrder_i[i] <= quicksum(x[i,j,k] for k in range(K))
for i in range(N)
for j in range(M))

#conflict
_ = model.addConstrs(
    Cbin[a,j] + Cbin[b,j] <= 1
for j in range(M)
for a, b in ConflictPair_alpha)

In [11]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 13624 rows, 18356 columns and 66213 nonzeros
Model fingerprint: 0xedd7d7f6
Variable types: 5200 continuous, 13156 integer (5330 binary)
Coefficient statistics:
  Matrix range     [3e-06, 4e+05]
  Objective range  [5e+00, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+05]
Presolve removed 3869 rows and 2865 columns
Presolve time: 0.17s
Presolved: 9755 rows, 15491 columns, 53014 nonzeros
Variable types: 4359 continuous, 11132 integer (3544 binary)
Found heuristic solution: objective 3.872829e+08
Found heuristic solution: objective 3.789255e+08

Root relaxation: objective 1.645192e+08, 5479 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6452e+08    0  357 3.7893e+08 1.

In [31]:
os.makedirs('solution', exist_ok=True)
solution_file_path = path.join(work_dir, 'solution', 'q2_solution.xlsx')

In [32]:
print('Express delivery')
with pd.ExcelWriter(solution_file_path, engine='xlsxwriter') as writer:
    df1 = pd.DataFrame([[int(x[i,j,0].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df1)
    df1.to_excel(writer, sheet_name='Express delivery')
    print('Air frieght')
    df2 = pd.DataFrame([[int(x[i,j,1].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df2)
    df2.to_excel(writer, sheet_name='Air frieght')
    print('Ocean frieght')
    df3 = pd.DataFrame([[int(x[i,j,2].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df3)
    df3.to_excel(writer, sheet_name='Ocean frieght')
    print('StockLevel')
    df = pd.DataFrame([[StockLevel[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='StockLevel')
    print('Shortage')
    df = pd.DataFrame([[Shortage[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='Shortage')

Express delivery


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,113,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Air frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,72,0,0,0,18,0,0,17,0,0,...,0,0,0,0,21,0,0,0,0,0
96,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ocean frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,97,155,213,134,77,206,108,76,195,...,0,0,0,0,0,0,0,0,0,0
4,0,67,49,99,36,11,112,246,112,59,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,224,109,150,0,32,80,260,33,179,0,...,74,183,238,0,172,0,62,0,0,0
96,0,0,0,0,0,0,0,269,0,0,...,384,0,0,0,151,0,279,0,0,0
97,0,0,0,0,160,37,81,199,0,110,...,222,0,0,118,0,157,229,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


StockLevel


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,662.0,607.0,435.0,241.0,147.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,410.0,357.0,289.0,104.0,91.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,346.0,167.0,166.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,208.0,258.0,180.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,365.0,303.0,220.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.3,0.3,0.3,0.3,0.3,0.3,1.3,0.3,0.3,...,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3
96,902.0,729.0,673.0,587.0,503.0,362.0,236.0,35.0,35.0,0.0,...,117.0,47.0,0.0,141.0,129.0,0.0,0.0,2.0,0.0,0.0
97,326.0,390.0,314.0,259.0,83.0,78.0,0.0,0.0,0.0,0.0,...,36.0,0.0,0.0,27.0,0.0,0.0,3.0,0.0,0.0,0.0
98,550.0,407.0,237.0,58.0,58.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shortage


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,0.00,0.000,0.0000,38.00000,78.000000,0.000000,152.00000,132.000000,...,66.000000,1.420000e+02,124.000000,0.000000,144.000000,131.000000,140.000000,131.000000,141.000000,0.000000
1,0.0,0.0,0.00,0.000,0.0000,45.00000,278.500000,329.950000,298.96500,446.275500,...,366.005483,4.622038e+02,481.542687,350.079881,315.055917,522.539142,365.777399,442.044179,423.430926,319.401648
2,0.0,0.0,0.00,0.000,82.0000,208.20000,112.820000,215.282000,100.52820,37.052820,...,86.428051,5.364281e+01,47.364281,202.736428,20.273643,107.027364,96.702736,73.670274,7.367027,196.736703
3,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.00000,0.000000,...,343.000000,4.350000e+02,699.000000,808.000000,808.000000,904.000000,1009.000000,1088.000000,1302.000000,1485.000000
4,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.00000,0.000000,...,361.000000,4.100000e+02,420.000000,420.000000,420.000000,534.000000,718.000000,830.000000,830.000000,845.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,17.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
96,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
97,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,1.968203e-12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
98,0.0,0.0,0.00,0.000,0.0000,119.00000,156.900000,114.690000,176.46900,153.646900,...,18.749945,2.748750e+02,98.487499,9.848750,232.984875,243.298487,24.329849,260.432985,96.043298,9.604330


In [33]:
print(f'TotalPurchaseCost    :{TotalPurchaseCost.getValue():.2f}')
print(f'TotalShipFixedCost   :{TotalShipFixedCost.getValue():.2f}')
print(f'TotalShipVarCost     :{TotalShipVarCost.getValue():.2f}')
print(f'TotalHoldingCost     :{TotalHoldingCost.getValue():.2f}')
print(f'TotalContainerCost   :{TotalContainerCost.getValue():.2f}')
print(f'TotalBackOrderCost   :{TotalBackOrderCost.getValue():.2f}')
print(f'TotalLostSaleCost    :{TotalLostSaleCost.getValue():.2f}')
print(f'TotalVendorFixedCost :{TotalVendorFixedCost.getValue():.2f}')
print(f'objective-value      :{model.ObjVal:.2f}')

TotalPurchaseCost    :111619350.00
TotalShipFixedCost   :3230.00
TotalShipVarCost     :377045.00
TotalHoldingCost     :3871377.75
TotalContainerCost   :1342000.00
TotalBackOrderCost   :5043084.79
TotalLostSaleCost    :42555681.58
TotalVendorFixedCost :36800.00
objective-value      :164848569.12
